<a href="https://colab.research.google.com/github/xslittlemaggie/Deep-Learning-Projects/blob/master/Save_%26_load_pre_trained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Course Project: CIFAR-10 & ResNet50 Transfer Learning </center></h1>

## Step 0: Import libraries

In [0]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras.utils import np_utils

from keras.applications.resnet50 import ResNet50, preprocess_input

import numpy as np
import random
% matplotlib inline
import matplotlib.pyplot as plt

# Confusion matrix result
from sklearn.metrics import classification_report, confusion_matrix

# visualizating of confusion matrix
import seaborn as sn
import pandas as pd

# upload new images
import urllib.request
import os
import cv2

Using TensorFlow backend.


## Step 1: Load cifar10 data

In [0]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)

print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


## Step 2: Data preprocessing

The pixel values are in the range of 0 to 255 for each of the red, green and blue channels.

It is good practice to work with normalized data.

We can easily normalize to range 0 to 1 by dividing each value by the maximum observation which is 255.

In [0]:
if K.image_data_format() == "channels_first":
  x_train = x_train.reshape(x_train.shape[0], 3, 32, 32)
  x_test = x_test.reshape(x_test.shape[0], 3, 32, 32)
  input_shape = (3, 32, 32)
  
else:
  x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
  x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)
  input_shape = (32, 32, 3)
  
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 225.
x_test /= 225.

# convert class vectors to binary class matrics
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

## Step 3: Load and use weights from a checkpoint ('weights.best.hdf5' trained before)

(During this process, I will upload the **'weights.best.hdf5'** manully from my local computer)

In [0]:
num_classes = 10

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) # Add dropout layer here

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) # Add dropout layer here

model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5)) # Add dropout layer here
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# load the pre-trained weights Manually (epochs = 100)
model.load_weights('weights.best.hdf5')
model.compile(loss = 'categorical_crossentropy', # better loss function for neural networks
            optimizer = 'adam', # Adam optimizer with 1.0e-4 learning rate
            metrics = ['accuracy'])
print('Created model and loaded weights from file')  

# load pima indians dataset
scores = model.evaluate(x_test, y_test, verbose = 0)
print('Test Acc: {}'.format(scores[1]))

W0803 08:30:28.723761 139955909515136 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0803 08:30:28.764647 139955909515136 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0803 08:30:28.771473 139955909515136 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0803 08:30:28.835353 139955909515136 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0803 08:30:28.836504 1399559095

Created model and loaded weights from file
Test Acc: 0.8239


So, the CNN model produces an test accuracy of 82%. 

Next, we will implement the transfer learning, and chek if we can improve the model. 

The ResNet50 model, pre-trained on the **imagenet weights** to implement transfer learning. (Other models including VGG16, VGG19, InceptiveV3 could also be implemented)

## Step 4: Import the ResNet50 Model to implement transfer learning

In [0]:
model = ResNet50(weights = 'imagenet', include_top = False, input_shape = (200, 200, 3))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step


The CIFAR10 dataset is small and similar to the 'ImageNet' dataset. So, we will remove the fully connected layers of the pre-trained network near the end. To implement this, we set 'include_top = False', while loading the ResNet50 model.

In [0]:
#pip install pillow
#pip install PIL

In [0]:
# Reshaping the training data
#x_train_new = np.array([cv2.resize(img_array, (200, 200)) for img_array in x_train[:10000]])('float32')


In [0]:
x_train_new = x_train_new.astype('float32')

In [0]:
resnet_train_input = preprocess_input(x_train_new)

In [0]:
train_features = model.predict(resnet_train_input)
np.savez('resnet_features_train', features=train_features)

KeyboardInterrupt: ignored

## Useful references
URL_1 = 'https://www.hackerearth.com/practice/machine-learning/transfer-learning/transfer-learning-intro/tutorial/'